# Лабораторная работа №5
### Задание 1
Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Выполнение работы:
### Шаг 1. Загрузка датасета

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm


df = pd.read_csv("./Base_min_shuffled.csv")

y = df["fraud_bool"]
X = df.drop("fraud_bool", axis=1)


columns_to_drop = ['payment_type', 'employment_status', 'source', 'device_os','housing_status']

# Удаление указанных столбцов
df = df.drop(columns=columns_to_drop)

y = df["fraud_bool"]
X = df.drop("fraud_bool", axis=1)

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### Метод решающих деревьев
Метод решающих деревьев (Decision Trees) представляет собой модель машинного обучения, которая используется для задач как классификации, так и регрессии. Он строит дерево решений путем разбиения данных на подмножества с учетом определенных условий по признакам. Каждый узел в дереве представляет собой тест на признаке, каждая ветвь - результат этого теста, а каждый лист - прогноз.

Гиперпараметры решающего дерева:
max_depth (максимальная глубина дерева):

Описание: Максимальная глубина дерева. Управляет тем, насколько глубоко дерево может разветвляться.
Возможные значения: Любое целое положительное число или None (без ограничения глубины).
min_samples_split (минимальное количество образцов для разделения узла):

Описание: Минимальное количество образцов, необходимых для разделения внутреннего узла. Большие значения помогают предотвратить переобучение.
Возможные значения: Любое целое положительное число.
min_samples_leaf (минимальное количество образцов в листе):

Описание: Минимальное количество образцов, необходимых для нахождения в листе. Большие значения помогают предотвратить переобучение.
Возможные значения: Любое целое положительное число.

In [10]:
param_grid_tree = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Создание модели решающего дерева
decision_tree = DecisionTreeClassifier()

# Поиск лучших параметров для решающего дерева
grid_search_tree = GridSearchCV(decision_tree, param_grid_tree, refit=True, verbose=3, cv=5)
grid_search_tree.fit(X_train, y_train)

# Лучшие параметры для решающего дерева
best_params_tree = grid_search_tree.best_params_
best_score_tree = grid_search_tree.best_score_

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.797 total time=   0.1s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.778 total time=   0.1s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.779 total time=   0.2s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.764 total time=   0.2s
[CV 5/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.781 total time=   0.1s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=0.795 total time=   0.1s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=0.786 total time=   0.1s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=0.787 total time=   0.1s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=5;, score=0.776 total time=   0.2s
[CV 5/5] END max_dep

### Вывод резулютатов обучения методом решающих деревьев 

In [11]:
best_tree = DecisionTreeClassifier(**best_params_tree)
best_tree.fit(X_train, y_train)

accuracy_tree = best_tree.score(X_test, y_test)
print("Лучшие параметры для решающего дерева:", best_params_tree)
print("Точность решающего дерева на тестовом наборе:", accuracy_tree)

predictions = best_tree.predict(X_test)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для решающего дерева: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10}
Точность решающего дерева на тестовом наборе: 0.8570833333333333

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.87      0.98      0.92      2038
           1       0.59      0.17      0.27       362

    accuracy                           0.86      2400
   macro avg       0.73      0.58      0.59      2400
weighted avg       0.83      0.86      0.82      2400



## Метод случайного леса
Метод случайного леса (Random Forest) является ансамблевым методом, который строит несколько деревьев решений во время обучения и объединяет их для получения более точных и стабильных прогнозов. Он использует технику "бутстрапа" для случайного выбора подмножества данных для обучения каждого дерева и также случайно выбирает подмножество признаков при каждом разделении узла.

### Гиперпараметры случайного леса:

### n_estimators (количество деревьев):

Описание: Количество деревьев в лесу.
Возможные значения: Любое целое положительное число.

### max_depth (максимальная глубина деревьев):

Описание: Максимальная глубина каждого дерева в лесу.
Возможные значения: Любое целое положительное число или None (без ограничения глубины).
min_samples_split (минимальное количество образцов для разделения узла):

Описание: Минимальное количество образцов, необходимых для разделения внутреннего узла. Большие значения помогают предотвратить переобучение.
Возможные значения: Любое целое положительное число.

### min_samples_leaf (минимальное количество образцов в листе):

Описание: Минимальное количество образцов, необходимых для нахождения в листе. Большие значения помогают предотвратить переобучение.
Возможные значения: Любое целое положительное число.

In [12]:
# Определение параметров для случайного леса
# Указал маленькие значения для ускорения обучения
param_grid_forest = {
    'n_estimators': range(2, 10),
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}

# Создание модели случайного леса
random_forest = RandomForestClassifier()

# Поиск лучших параметров для случайного леса
grid_search_forest = GridSearchCV(random_forest, param_grid_forest, refit=True, verbose=3, cv=5)
grid_search_forest.fit(X_train, y_train)

# Лучшие параметры для случайного леса
best_params_forest = grid_search_forest.best_params_
best_score_forest = grid_search_forest.best_score_

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2;, score=0.833 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2;, score=0.828 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2;, score=0.830 total time=   0.0s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2;, score=0.833 total time=   0.0s
[CV 5/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=2;, score=0.820 total time=   0.0s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=3;, score=0.819 total time=   0.0s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=3;, score=0.828 total time=   0.0s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=3;, score=0.808 total time= 

## Результаты обучения с помощью метода случайного леса

In [13]:
best_forest = RandomForestClassifier(**best_params_forest)
best_forest.fit(X_train, y_train)

accuracy_forest = best_forest.score(X_test, y_test)
print("Лучшие параметры для случайного леса:", best_params_forest)
print("Точность случайного леса на тестовом наборе:", accuracy_forest)

predictions = best_forest.predict(X_test)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для случайного леса: {'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 9}
Точность случайного леса на тестовом наборе: 0.8604166666666667

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      2038
           1       0.58      0.26      0.36       362

    accuracy                           0.86      2400
   macro avg       0.73      0.61      0.64      2400
weighted avg       0.84      0.86      0.84      2400

